In [9]:
import web3 
import math
from sklearn.datasets import fetch_california_housing
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
import warnings
from UniswapV3TradingFramework import priceToSqrt,V3TraderFramework
from position import Position
from swap import Swap 
warnings.filterwarnings('ignore', category=UserWarning)
import yfinance as yf 
import talib 
import requests 

# WBTC/USDC Pool Setup 

In [7]:
pool_address = "0x99ac8cA7087fA4A2A1FB6357269965A2014ABc35".lower()
fee_tier = 0.003
graph_url = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"
api_key = "355571043fb273821d416bb76ef0d85c"
v3_trader = V3TraderFramework(url=graph_url, pool_address = pool_address, poolFeeTier=fee_tier, api_key=api_key)



# Data Params

In [13]:

batch_size = 1000
start_ts = 1747238400
end_ts = 1749916800
frequency = 864000



# CEX Data Collection

In [28]:
btc_price_data = []

current_ts = start_ts+frequency 

total_orders = 0

while current_ts < end_ts:
    
    response = requests.get('https://data-api.coindesk.com/spot/v1/historical/days',
    params={"market":"kraken","instrument":"BTC-USD","limit":{batch_size},
            "aggregate":1,"fill":"true","apply_mapping":"true","response_format":"JSON","to_ts":{current_ts}},
    headers={"Content-type":"application/json; charset=UTF-8"}
    )
    
    raw_data = response.json()
    price_data = raw_data['Data']
    for price_order in price_data:    
        btc_price_order = {
            'Open': price_order['OPEN'],
            'High':price_order['HIGH'],
            'Low':price_order['LOW'],
            'Close':price_order['CLOSE'],
            'Volume':price_order['VOLUME'],
            'Timestamp':price_order['TIMESTAMP']
        }
    
        btc_price_data.append(btc_price_order)
    current_ts += frequency
    total_orders += batch_size
    print("Total Orders Fetched: " + str(total_orders))


btc_price_data = pd.DataFrame(btc_price_data)


btc_price_data['Returns'] = np.asarray(btc_price_data['Close'].pct_change())
btc_price_data['5 Day RSI'] = talib.RSI(btc_price_data['Close'],timeperiod=5) 
btc_price_data['5 Day ADX'] = talib.ADX(btc_price_data['High'],btc_price_data['Low'],btc_price_data['Close'],timeperiod=5) 
btc_price_data['5 Day ROC'] = talib.ROC(btc_price_data['Close'], timeperiod=5)
btc_price_data['OBV'] = talib.OBV(btc_price_data['Close'], btc_price_data['Volume'])
btc_price_data['5 Day OBV EMA'] = talib.EMA(btc_price_data['OBV'], timeperiod=5)
btc_price_data['5 Day SMA'] = talib.SMA(btc_price_data['Close'], timeperiod=5)
btc_price_data['5 Day EMA'] = talib.EMA(btc_price_data['Close'], timeperiod=5)
btc_price_data = btc_price_data.dropna()
btc_price_data.index = btc_price_data['Timestamp']


Total Orders Fetched: 100
Total Orders Fetched: 200
Total Orders Fetched: 300


,Open,High,Low,Close,Volume,Timestamp,Returns,5 Day RSI,5 Day ADX,5 Day ROC,OBV,5 Day OBV EMA,5 Day SMA,5 Day EMA
Timestamp,,,,,,,,,,,,,,
1740268800,96600.1,96621.6,95300.0,96303.4,359.687588,1740268800,-0.003071,40.733252,42.921375,0.735774,-549.454868,-37.401216,96793.56,96589.113745
1740355200,96303.4,96483.0,91400.0,91534.4,2273.508327,1740355200,-0.049521,17.053555,49.504525,-5.269095,-2822.963195,-965.921875,95775.30,94904.209163
1740441600,91534.4,92487.3,86000.0,88632.6,5210.263096,1740441600,-0.031702,11.825028,57.153044,-9.870438,-8033.226291,-3321.690014,93834.00,92813.672775
1740528000,88632.6,89271.0,80525.0,84089.8,5878.701032,1740528000,-0.051254,7.390774,64.214098,-12.497151,-13911.927322,-6851.769117,91432.06,89905.715184
1740614400,84089.8,86951.2,82611.0,84691.4,2506.417961,1740614400,0.007154,12.803448,69.862942,-12.327834,-11405.509362,-8369.682532,89050.32,88167.610122


# DEX Data Collection


In [ ]:
adjusted_start_ts = 1740268800
adjusted_end_ts = 1749772800

In [ ]:
swaps = v3_trader.retrieveSwapsData(from_time=adjusted_start_ts, to_time=adjusted_end_ts,batch_size=batch_size, frequency=frequency)